In [ ]:
import os
import numpy as np

from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions

from resnet50 import ResNet50
from resnet50 import preprocess_input as res_preprocess_input

from inception_v3 import InceptionV3
from inception_v3 import preprocess_input as incep_preprocess_input

In [ ]:
def download_meal_images(num_images=100, destination='./images/'):

    import boto3
    """
    REQUIRES CORRECT S3 CREDENTIALS IN YOU ~/.aws/credentials file
    """
    bucket_name = 'filepicker-images-prevent'
    meal_words = ['lunch', 'dinner', 'breakfast', 'snack'] # images have one of these words in their filename

    # connect
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)

    downloaded = []
    for key in bucket.objects.all():

        if any(meal_word in key.key for meal_word in meal_words):
            dest = os.path.join(destination, key.key)
            s3.meta.client.download_file(bucket_name, key.key, dest)
            downloaded.append(dest)

        if len(downloaded) > num_images:
            return downloaded

In [ ]:
new_images = download_meal_images(3)

In [ ]:
model_lib = {
    'inception': {
        'model': InceptionV3(include_top=True, weights='imagenet'),
        'preprocessor': incep_preprocess_input,
        'target_size': (299, 299)
    },
    'resnet': {
        'model': ResNet50(include_top=True, weights='imagenet'),
        'preprocessor': res_preprocess_input,
        'target_size': (224, 224)
    }
}

In [ ]:
# images = ['./images/' + f for f in os.listdir('./images') if f != '.DS_Store']

In [ ]:
def get_predictions(image_path, models):
    predictions = {}
    for model_name in models:
        # load model
        model = model_lib[model_name]['model']
        
        # process image
        img = image.load_img(image_path, target_size=model_lib[model_name]['target_size'])
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        preprocessor = model_lib[model_name]['preprocessor']
        x = preprocessor(x)
        
        # predict
        preds = model.predict(x)
        decoded = decode_predictions(preds)
              
        # store predictions for future fun and profit
        predictions[model_name] = decoded[0]
    return img, predictions

In [ ]:
im_number = 1

In [ ]:
img, preds = get_predictions(new_images[im_number], models=model_lib.keys())

In [ ]:
img

In [ ]:
preds

In [ ]:
from IPython.display import Image
Image(filename=new_images[im_number]) 